In [0]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("fill_na").getOrCreate()
sc=spark.sparkContext
rdd=sc.textFile("/FileStore/tables/fill_na-1.txt")
# Remove empty lines from the RDD
filtered_rdd = rdd.filter(lambda line: line.strip() != '')

print(filtered_rdd.collect())

['spark spark spark spark spark', 'spark spark spark spark ', 'spark spark spark', 'spark spark', 'spark']


In [0]:
rdd1=filtered_rdd.map(lambda x : (str)(x)).map(lambda x: x.split(' '))
rdd2=rdd1.map(lambda x : len(x))

max_length = rdd2.max()

In [0]:
def fillMissingValues(x, max_length):
    results = []
    for i in range(max_length):
        try:
            results.append(x[i])
        except IndexError:
            results.append('nodata')
    return results

In [0]:
# Apply the function to fill missing values
rdd4 = rdd1.map(lambda x: fillMissingValues(x, max_length))

# Display the final RDD
print(rdd4.collect())

[['spark', 'spark', 'spark', 'spark', 'spark'], ['spark', 'spark', 'spark', 'spark', ''], ['spark', 'spark', 'spark', 'nodata', 'nodata'], ['spark', 'spark', 'nodata', 'nodata', 'nodata'], ['spark', 'nodata', 'nodata', 'nodata', 'nodata']]


In [0]:
df=rdd1.toDF(['line'])
df1=df.fillna('nodata').show()


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1916403851493285>:2
      1 df=rdd1.toDF(['line'])
----> 2 df1=df.fillna('nodata').show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916         message_parameters={"arg_name": "vertical", "arg_type": type(vertical).__name__},
    917     )
    919 if isinstance(truncate, bool) and truncate:
--> 920     print(

In [0]:
data=spark.read.csv("/FileStore/tables/fill_na-1.txt")
data.show(truncate=0)

+-----------------------------+
|_c0                          |
+-----------------------------+
|spark spark spark spark spark|
|spark spark spark spark      |
|spark spark spark            |
|spark spark                  |
|spark                        |
+-----------------------------+



In [0]:
fill=data.fillna("nodata")
fill.show(truncate=0)

+-----------------------------+
|_c0                          |
+-----------------------------+
|spark spark spark spark spark|
|spark spark spark spark      |
|spark spark spark            |
|spark spark                  |
|spark                        |
+-----------------------------+

